In [77]:
%reset -f
from parflow import Run
from parflow.tools.fs import get_absolute_path
from parflow.tools.io import write_pfb, read_pfb
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch.utils.data import TensorDataset
from utils import init_location, cal_z_locid, cal_velz_loc, \
                  interpolate_vel, data_normalize, To_Tensor
# from parflow.tools.hydrology import calculate_water_table_depth, \
# calculate_overland_flow_grid, calculate_overland_fluxes
# %matplotlib widget

In [78]:
p_start = 35040
t_start = 0 #iteration start
t_end = 10 #iteration end
real_path = 'False'
selectd = np.array([0,20,0,5,0,20])

npcell = 40  #used for train
testsize = 4  #used for test
npcell = npcell + testsize  #all
# how many particles in a grid cell

interval = 1 #always no gap
step_size = 1 #always one hour

In [79]:
# input model parameters
nx = 20
ny = 5
nz = 20

dx = 5
dy = 0.2
dz = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, \
               0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.3, 0.1])

boundings = np.array([0., 100., 0., 1., 0., 9.4])

In [80]:
for ii in range(t_start,t_end):
    
    istep = 1 + ii #current iterate step
    print('istep',istep)

    Vx = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.velx.' \
                                    +str(istep+p_start).zfill(5)+'.pfb'))
    Vy = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.vely.' \
                                    +str(istep+p_start).zfill(5)+'.pfb'))
    Vz = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.velz.' \
                                    +str(istep+p_start).zfill(5)+'.pfb'))
    Saturation = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.satur.' \
                                            +str(istep+p_start).zfill(5)+'.pfb'))
    Porosity = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.porosity.pfb'))
    # print(Vx.shape, Vy.shape, Vz.shape)

    if real_path == 'False' or (real_path == 'True' and istep == (t_start + 1)):
        
        #setup initial location
        initx,inity,initz = init_location(npcell,nx,ny,nz,dx,dy,dz)
        # initz[1,0,:,:]

        #cal locid
        Px = np.copy(initx)
        Py = np.copy(inity)
        Pz = np.copy(initz)
        
        print('real_path',real_path)
    
    if real_path == 'True' and istep != (t_start + 1):
        
        initx = np.copy(Px)
        inity = np.copy(Py)
        initz = np.copy(Pz)
        
        print('real_path',real_path)

    Plocx = np.floor(Px/dx)
    Plocy = np.floor(Py/dy)
    Plocz = cal_z_locid(npcell,nx,ny,nz,dz,Pz)

    # cal vel loc
    # this can be got by random number directly
    # but for multiple steps, we still need to calculate
    # so we provide the template here
    # we may also calculate it from random number
    # i mean like add the intial fraction with the new_distance/vel
    # but additional other calculation may need

    Clocx = (Px - Plocx*dx)/dx
    Clocy = (Py - Plocy*dy)/dy
    Clocz = cal_velz_loc(npcell,nx,ny,nz,dz,Pz,Plocz)

    Vpx, Vpy, Vpz = interpolate_vel(npcell,nx,ny,nz,Plocx,Plocy,Plocz,Clocx,Clocy,Clocz, \
                                   Vx,Vy,Vz,Saturation,Porosity)

    Px[:,selectd[0]:selectd[1],selectd[2]:selectd[3],selectd[4]:selectd[5]] = \
    Px[:,selectd[0]:selectd[1],selectd[2]:selectd[3],selectd[4]:selectd[5]] + \
    Vpx[:,selectd[0]:selectd[1],selectd[2]:selectd[3],selectd[4]:selectd[5]]
    
    Py[:,selectd[0]:selectd[1],selectd[2]:selectd[3],selectd[4]:selectd[5]] = \
    Py[:,selectd[0]:selectd[1],selectd[2]:selectd[3],selectd[4]:selectd[5]] + \
    Vpy[:,selectd[0]:selectd[1],selectd[2]:selectd[3],selectd[4]:selectd[5]]
    
    Pz[:,selectd[0]:selectd[1],selectd[2]:selectd[3],selectd[4]:selectd[5]] = \
    Pz[:,selectd[0]:selectd[1],selectd[2]:selectd[3],selectd[4]:selectd[5]] + \
    Vpz[:,selectd[0]:selectd[1],selectd[2]:selectd[3],selectd[4]:selectd[5]]

    tt,initx1,inity1,initz1,Px1,Py1,Pz1 = data_normalize(istep,interval,step_size,t_start,t_end, \
                                                   initx,inity,initz,Px,Py,Pz,boundings)

    start_train_new, end_train_new, time_train_new = To_Tensor(0,npcell-testsize, \
               selectd[0],selectd[2],selectd[4],selectd[1],selectd[3],selectd[5], \
                                                initx1,inity1,initz1,Px1,Py1,Pz1,tt)
    start_test_new, end_test_new, time_test_new = To_Tensor(npcell-testsize,npcell, \
               selectd[0],selectd[2],selectd[4],selectd[1],selectd[3],selectd[5], \
                                                initx1,inity1,initz1,Px1,Py1,Pz1,tt)
    
    if istep == t_start + 1:
        start_train = start_train_new
        end_train   = end_train_new
        time_train  = time_train_new
        
        start_test  = start_test_new
        end_test    = end_test_new
        time_test   = time_test_new
        
    else:
        
        start_train = torch.cat((start_train, start_train_new),dim=0)
        end_train   = torch.cat((end_train, end_train_new),dim=0)
        time_train  = torch.cat((time_train, time_train_new),dim=0)
        
        start_test  = torch.cat((start_test, start_test_new),dim=0)
        end_test    = torch.cat((end_test, end_test_new),dim=0)
        time_test   = torch.cat((time_test, time_test_new),dim=0)
    

istep 1
real_path False
out of boundary: -0.9458742157331939 -0.1899927325764904 0.9999382600476601 -0.9459372875341384 -0.18999273257976323 1.0023026577847318
out of boundary: 1
out of boundary: -0.9249709798235157 -0.9314093053672268 0.9977632466966686 -0.925058409864905 -0.9314093053657393 1.0001251058056821
out of boundary: 2
out of boundary: -0.9676408945223222 -0.7788563974919596 0.9982320474807669 -0.9676786020376886 -0.7788563974871638 1.0005944538169729
out of boundary: 3
out of boundary: -0.9107305235984406 0.304075075692267 0.9987899651792624 -0.910834548275327 0.3040750756923858 1.001153022383071
out of boundary: 4
out of boundary: -0.93666183235532 -0.2816431604145053 0.9990257989464497 -0.9367356395817998 -0.2816431604154729 1.0013891314252574
out of boundary: 5
out of boundary: -0.9549819496980171 -0.79062165064454 0.9991882663108849 -0.9550344084665296 -0.7906216506399995 1.0015517888332481
out of boundary: 6
out of boundary: -0.9163853489253301 -0.14072525899384403 0.9

In [81]:
print(start_train.shape,start_test.shape,end_train.shape,end_test.shape)

torch.Size([799778, 3]) torch.Size([79974, 3]) torch.Size([799778, 3]) torch.Size([79974, 3])


In [82]:
data_train = TensorDataset(start_train,end_train,time_train)
data_test = TensorDataset(start_test,end_test,time_test)

if real_path == 'True':
    torch.save(data_train,'data_check.pth')  
else:
    torch.save(data_train,'data_train.pth')
    torch.save(data_test,'data_test.pth')

In [83]:
count = 0
for x in data_train:#generator
    print(x)
    count = count + 1
    if count > 1000: break

(tensor([-0.9704, -0.9560, -0.9381]), tensor([-0.9704, -0.9560, -0.9381]), tensor([-0.8000]))
(tensor([-0.8754, -0.8662, -0.9611]), tensor([-0.8755, -0.8662, -0.9611]), tensor([-0.8000]))
(tensor([-0.7352, -0.9265, -0.9490]), tensor([-0.7353, -0.9265, -0.9490]), tensor([-0.8000]))
(tensor([-0.6863, -0.7489, -0.9499]), tensor([-0.6864, -0.7489, -0.9499]), tensor([-0.8000]))
(tensor([-0.5178, -0.8557, -0.9290]), tensor([-0.5179, -0.8557, -0.9290]), tensor([-0.8000]))
(tensor([-0.4660, -0.7611, -0.9205]), tensor([-0.4661, -0.7611, -0.9205]), tensor([-0.8000]))
(tensor([-0.3927, -0.9164, -0.9713]), tensor([-0.3928, -0.9164, -0.9713]), tensor([-0.8000]))
(tensor([-0.2252, -0.7050, -0.9021]), tensor([-0.2253, -0.7050, -0.9021]), tensor([-0.8000]))
(tensor([-0.1392, -0.9401, -0.9223]), tensor([-0.1393, -0.9401, -0.9223]), tensor([-0.8000]))
(tensor([-0.0532, -0.7127, -0.9335]), tensor([-0.0533, -0.7127, -0.9335]), tensor([-0.8000]))
(tensor([ 0.0944, -0.7017, -0.9278]), tensor([ 0.0943, -0.70

In [84]:
b = data_train[:][0]
a = b.numpy()

b = data_train[:][1]
c = b.numpy()

b = data_train[:][2]
t = b.numpy()

In [85]:
# import time
# fig = plt.figure()
ax = plt.axes(projection='3d')

# Data for a three-dimensional line
# for ii in range(600):
ii =10
xline = c[ii:6000:600,0]
yline = c[ii:6000:600,1]
zline = c[ii:6000:600,2]
xline0 = a[ii:6000:600,0]
yline0 = a[ii:6000:600,1]
zline0 = a[ii:6000:600,2]
tdata = t[ii:6000:600]
ax.plot3D(xline, yline, zline, 'gray')
# ax.plot3D(xline0, yline0, zline0, 'blue')
ax.scatter3D(xline, yline, zline, c = tdata, cmap='Reds')
# time.sleep(2)
# ax.show()